<a href="https://colab.research.google.com/github/naychelynn/Notebooks_Kaggle/blob/main/BIQA_ELM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tqdm import tqdm
import time
import pandas as pd
from tensorflow.keras.utils import load_img
from skimage.restoration import estimate_sigma
from skimage import img_as_float
from scipy import fftpack
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def im2patch(im, pch_size, stride=1):
    '''
    Reference of this part:
    Implementation of noise level estimation of the following paper:
    Chen G , Zhu F , Heng P A . An Efficient Statistical Method for Image Noise Level Estimation[C]// 2015 IEEE International Conference
    on Computer Vision (ICCV). IEEE Computer Society, 2015.

    Transform image to patches.
    Input:
        im: 3 x H x W or 1 X H x W image, numpy format
        pch_size: (int, int) tuple or integer
        stride: (int, int) tuple or integer
    '''
    if isinstance(pch_size, tuple):
        pch_H, pch_W = pch_size
    elif isinstance(pch_size, int):
        pch_H = pch_W = pch_size
    else:
        print('The input of pch_size must be a integer or a int tuple!')

    if isinstance(stride, tuple):
        stride_H, stride_W = stride
    elif isinstance(stride, int):
        stride_H = stride_W = stride
    else:
        print('The input of stride must be a integer or a int tuple!')

    stride_H, stride_W = 1 , 1
    # print(stride_W)                                   # 3
    C, H, W = im.shape
    num_H = len(range(0, H-pch_H+1, stride_H))          # 169 | 377
    num_W = len(range(0, W-pch_W+1, stride_W))          # 126 | 505
    num_pch = num_H * num_W
    # print("num_H")
    # print(num_H)
    # print("num_W")
    # print(num_W)
    pch = np.zeros((C, pch_H*pch_W, num_pch), dtype=im.dtype)
    kk = 0
    for ii in range(pch_H):
        for jj in range(pch_W):
            temp = im[:, ii:H-pch_H+ii+1:stride_H, jj:W-pch_W+jj+1:stride_W]
            pch[:, kk, :] = temp.reshape((C, num_pch))
            kk += 1

    return pch.reshape((C, pch_H, pch_W, num_pch))


##################################################

    def noise_estimate(im, pch_size=8):
    '''
    Input:
        im: the noise image, H x W x 3 or H x W numpy tensor, range [0,1]
        pch_size: patch_size
    Output:
        noise_level: the estimated noise level
    '''

    if im.ndim == 3:
        im = im.transpose((2, 0, 1))
    else:
        im = np.expand_dims(im, axis=0)

    # image to patch
    pch = im2patch(im, pch_size, 3)     # C x pch_size x pch_size x num_pch tensor

    num_pch = pch.shape[3]              # number of patches > ..... | .....
    pch = pch.reshape((-1, num_pch))    # d x num_pch matrix
    # print(pch.shape)                  # (3*8*8, ..... ) > (192, ..... )
    d = pch.shape[0]                    # 192
    mu = pch.mean(axis=1, keepdims=True)    # d x 1 : find mean  # (192, 1)
    X = pch - mu                        # how much varied from mean
    sigma_X = np.matmul(X, X.transpose()) / num_pch # matrix multiplication and divide by no of patch
    # print(sigma_X.shape)              #  (192, 192)
    sig_value, _ = np.linalg.eigh(sigma_X) #  (192,~)
    # sig_value.sort()                    # sort eigen value array in ascending order

    for ii in range(-1, -d-1, -1):      # order array in negative sequences
        tau = np.mean(sig_value[:ii])   # find mean of eigen values  192 .. 191 .. mean
        # return sqrt of pixel if greater than mean or less than mean
        if np.sum(sig_value[:ii]>tau) == np.sum(sig_value[:ii] < tau):
            return np.sqrt(tau)

In [ ]:
if __name__ == '__main__':
    image_paths, all_est_noise_level, all_sigma_est, all_entropy_est,  = [], [], [], []

    img_path = 'E:/... /01_Gaussian_TID2013/'
    df = pd.read_csv('E:/.../'+'01_Gaussian_TID2013.csv')
    print("05_Gaussian_TID2013 with Norm >>>>>>>>>>>>>>>>>>>>>>>>>>>")

    for filename in tqdm(os.listdir(img_path)):
        temp = filename.split(' ')
        image_path = os.path.join(img_path, temp[0])
        image_paths.append(image_path)


    df['Image'] = image_paths
    df['Score'] = df['Score'].astype(float)

    for image in tqdm(df['Image']):
        img = load_img(image, grayscale=False)
        im = img_as_float(img)
        start = time.time()

        ####### extraction of feature 1
        est_noise_level = noise_estimate(im, 8)

        all_est_noise_level.append(est_noise_level*255)

        ####### extraction of feature 2
        # take the fourier transform of the image
        F1 = fftpack.fft2(img)
        # Shift the quadrants around so that low spatial frequencies are in
        # the center of the 2D fourier transformed image.
        F2 = fftpack.fftshift(F1)
        # Calculate a 2D power spectrum of the Fourier Transform
        spectrum = np.abs(F2)**2
        # calculate spectrum entropy value from Power Spectrum
        spectral_entropy =  stats.entropy(spectrum[:, 1])

        ####### extraction of feature 3
        # Estimate the average noise standard deviation across color channels.
        sigma_est = estimate_sigma(im, average_sigmas=True, multichannel=True)


        entropy_est = np.mean(spectral_entropy)         # Spectral entropy
        all_sigma_est.append(sigma_est)
        all_entropy_est.append(entropy_est)

        end = time.time()
        time_elapsed = end - start

In [ ]:
t=np.vstack((np.array(all_sigma_est), np.array(all_est_noise_level),np.array(all_entropy_est)));
irx=np.transpose(t); iry=df['Score'].tolist()

pearsonr_corr_1 = stats.pearsonr(np.squeeze(all_sigma_est), np.squeeze(all_est_noise_level))
spearmanr_corr_1 = stats.spearmanr(np.squeeze(all_sigma_est), np.squeeze(all_est_noise_level))
print("\nPLCC 1: ", pearsonr_corr_1[0], " | SROCC 1: ", spearmanr_corr_1[0])
all_pearsonr_corr_1.append(pearsonr_corr_1[0])
all_spearmanr_corr_1.append(spearmanr_corr_1[0])

pearsonr_corr_2 = stats.pearsonr(np.squeeze(all_est_noise_level), np.squeeze(all_entropy_est))
spearmanr_corr_2 = stats.spearmanr(np.squeeze(all_est_noise_level), np.squeeze(all_entropy_est))
print("\nPLCC 2: ", pearsonr_corr_2[0], " | SROCC 2: ", spearmanr_corr_2[0])
all_pearsonr_corr_2.append(pearsonr_corr_2[0])
all_spearmanr_corr_2.append(spearmanr_corr_2[0])

pearsonr_corr_3 = stats.pearsonr(np.squeeze(all_entropy_est), np.squeeze(all_sigma_est))
spearmanr_corr_3 = stats.spearmanr(np.squeeze(all_entropy_est), np.squeeze(all_sigma_est))
print("\nPLCC 3: ", pearsonr_corr_3[0], " | SROCC 3: ", spearmanr_corr_3[0])
all_pearsonr_corr_3.append(pearsonr_corr_3[0])
all_spearmanr_corr_3.append(spearmanr_corr_3[0])

#**************************
irx, iry = np.array(irx), np.array(iry)
all_Pear, all_Spear, all_time = [], [], []

#%
x_train, x_test, y_train, y_test = train_test_split(irx, iry, test_size=0.1)
# build model and train
model = elm.elm(hidden_units=500, activation_function='relu', random_type='normal', x=x_train, y=y_train, C=0.1, elm_type='reg')
print('/n>>> L1 Regularization 500 hidden units C=0.1 Entropy >>>')
# beta, train_accuracy, running_time = model.fit('no_re')
beta, train_accuracy, running_time = model.fit('solution1')
# beta, train_accuracy, running_time = model.fit('solution2')
print('regressor running time:', running_time)
all_time.append(running_time)

In [ ]:
feature_names = ['Spatial Var', 'Fourier Entrp', 'Wavelet Std']
forest = RandomForestRegressor(random_state=0)
forest.fit(x_train, y_train)

start_time = time.time()
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

forest_importances = pd.Series(importances, index=feature_names)
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
plt.xticks(rotation=30)
ax.set_title("Noise Statstical Feature Importance using Mean decrease in impurity")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()
start_time = time.time()
result = permutation_importance(
    forest, x_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)
elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importance: {elapsed_time:.3f} seconds")


fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=None, ax=ax)
plt.xticks(rotation=30)
ax.set_title("Noise Statstical Feature Importance using Permutation")
ax.set_ylabel("Mean accuracy")
fig.tight_layout()
plt.show()

In [ ]:
# test
prediction = model.predict(x_test)
yt1, yt2 = [], []
for i in range(len(prediction)):
    x, y = y_test[i], prediction[i]
    nas = np.logical_or(np.isnan(x), np.isnan(y))
    if not nas:
        yt1.append(x)
        yt2.append(y)

#%
pearsonr_score = stats.pearsonr(np.squeeze(yt1), np.squeeze(yt2))
spearmanr_score = stats.spearmanr(np.squeeze(yt1), np.squeeze(yt2))
print("\nPearson Correlation: ", pearsonr_score[0], " | Spearman Correlation: ", spearmanr_score[0])
all_Pear.append(pearsonr_score[0])
all_Spear.append(spearmanr_score[0])